[![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/shhommychon/pytorch_semantic_segmentation/blob/main/_jupyternotebooks/ngrok_test_env.ipynb)

# [ngrok](https://ngrok.com/) 세팅

In [ ]:
NGROK_TOKEN = "hello_hommy"
PASSWORD = "hello_hommy"

In [ ]:
# from google.colab import drive
# drive.mount('/content/drive/')

In [ ]:
# Install colab_ssh on google colab
!pip install colab_ssh --upgrade

from colab_ssh import launch_ssh
launch_ssh(NGROK_TOKEN, PASSWORD)

# 테스트 코드

## 환경 설정

In [ ]:
!git clone https://github.com/shhommychon/pytorch_semantic_segmentation.git

In [ ]:
import sys
sys.path.append("/content/pytorch_semantic_segmentation")
import pytorch_semantic_segmentation.semantic_segmentation as myss

In [ ]:
import torch
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F

print (f"PyTorch version: {torch.__version__}")

In [ ]:
print(f"cuda available? : {torch.cuda.is_available()}")
_device_count = torch.cuda.device_count()
print(f"cuda device count? : {_device_count}")
print()

device = torch.device(f"cuda:{_device_count-1}" if torch.cuda.is_available() else "cpu")
print (f"device: {device}")